In [ ]:
!pip install gdal
!sudo apt-get update
!sudo apt-get install
!sudo apt-get install
!pip install Pillow
!sudo apt-get install -y gdal-bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [27.8 kB]
Get:13 https://ppa.launch

In [ ]:
from osgeo import gdal, osr
import time
from PIL import Image
import numpy as np
!gdalinfo --version
import os
import shutil
import geopandas as gpd
from shapely.geometry import box
import re
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

GDAL 3.6.4, released 2023/04/17


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Função para recortar imagens
def recortar_imagens(input_folder, coordenadas, output_folder, size=256):
    # Abre a imagem de entrada
    dataset = gdal.Open(input_folder)
    transform = dataset.GetGeoTransform()

    for index, (x, y) in enumerate(coordenadas):
        # Converte coordenadas de projeção para coordenadas de pixel
        col = int((x - transform[0]) / transform[1])
        row = int((y - transform[3]) / transform[5])

        # Calcula o offset para centrar o recorte no ponto de interesse
        offset_x = col - size // 2
        offset_y = row - size // 2

        # Define o nome do arquivo de saída para o recorte atual
        output_file = f"{output_folder}/imagem_{coordenadas[index][0]}L_{coordenadas[index][1]}N.tif"

        # Usa gdal_translate para realizar o recorte, saída de dados Int8
        options = gdal.TranslateOptions(format='GTiff', outputType=gdal.GDT_Byte, srcWin=[offset_x, offset_y, size, size])
        gdal.Translate(output_file, input_folder, options=options)

        print(f"Imagem recortada {index} salva em: {output_file}")
        time.sleep(0.5)


In [ ]:
!pip install geopandas shapely

In [ ]:
# Função para extrair coordenadas donome
def extrair_coordenadas_do_nome(nome_arquivo):
    padrao = re.compile(r"imagem_(-?\d+)L_(-?\d+)N.tif")
    correspondencia = padrao.search(nome_arquivo)
    if correspondencia:
        x, y = map(int, correspondencia.groups())  # Garante que x e y sejam inteiros
        return x, y
    else:
        return None

In [ ]:
def recortar_shapefile_por_imagens(input_shapefile, input_image_folder, output_folder, size=256):
    # Carrega o shapefile de desmatamento
    gdf_desmatamento = gpd.read_file(input_shapefile)

    # Lista todas as imagens na pasta especificada
    image_files = [file for file in os.listdir(cropped_images) if file.endswith('.tif')]

    for image_file in image_files:
      try:
        coordenadas_nome = extrair_coordenadas_do_nome(image_file)
        if coordenadas_nome:
            x, y = coordenadas_nome
            image_path = os.path.join(input_image_folder, image_file)

            # Abre a imagem de entrada com GDAL
            dataset = gdal.Open(image_path)
            transform = dataset.GetGeoTransform()

            # Converte coordenadas de projeção para coordenadas de pixel e volta
            col = int((x - transform[0]) / transform[1])
            row = int((y - transform[3]) / transform[5])
            x_min = transform[0] + transform[1] * (col - size // 2)
            x_max = transform[0] + transform[1] * (col + size // 2)
            y_max = transform[3] + transform[5] * (row - size // 2)
            y_min = transform[3] + transform[5] * (row + size // 2)

            # Cria um polígono representando a área do recorte
            poligono_recorte = box(x_min, y_min, x_max, y_max)

            # Realiza o recorte
            gdf_recortado = gdf_desmatamento.clip(poligono_recorte)

            # Define o nome do arquivo de saída baseado no nome da imagem
            output_shapefile_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.shp')

            # Salva o resultado
            gdf_recortado.to_file(output_shapefile_path)

            print(f"Shapefile recortado salvo em: {output_shapefile_path}")
        else:
            print(f"Não foi possível extrair as coordenadas do nome do arquivo: {image_file}")
      except Exception as e:
        print(f"Erro ao processar o arquivo {image_file}: {e}")

In [ ]:
def verifica_coordenada_vertice(input_file_image):
  # Verifica as coordenadas dos vertices
  dataset = gdal.Open(input_file_image)
  transform = dataset.GetGeoTransform()
  x_min = transform[0]
  y_max = transform[3]
  x_max = x_min + (transform[1] * dataset.RasterXSize)
  y_min = y_max + (transform[5] * dataset.RasterYSize)  # transform[5] é negativo
  print("Upper Left (xmin, ymax): ", (x_min, y_max))
  print("Lower Right (xmax, ymin): ", (x_max, y_min))
  return x_min, y_min, x_max, y_max

In [ ]:
path = '/content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0'

#Diretórios de Imagens de Entrada, Saída e arquivos shapefile
main_images = path + '/main_images/AMAZONIA_1_WFI_20230724_037_017_L4.tif' # Coordenada 1
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230629_037_017_L4.tif' # Coordenada 2
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230525_037_017_L4.tif' # Coordenada 3
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230704_037_017_L4.tif' # Coordenada 4
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230709_037_017_L4.tif' # Coordenada 5


#main_images = path + '/main_images/AMAZONIA_1_WFI_20230701_038_017_L4.tif' # Coordenada 6
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230726_038_017_L4.tif' # Coordenada 7
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230810_038_017_L4.tif' # Coordenada 8
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230820_038_017_L4.tif' # Coordenada 9
#main_images = path + '/main_images/AMAZONIA_1_WFI_20230616_038_017_L4.tif' # Coordenada 10



shp_deforestation_total = path +'/shapefiles_total/AMAZONIA_1_WFI_20230724_037_017_L4.shp'
#shp_deforestation_total = path +'/shapefiles_total/AMAZONIA_1_WFI_20230701_038_017_L4.shp'

cropped_images = path + '/cropped_images'
shp_deforestation = path + '/shapefiles_deforestation'

In [ ]:
coordenadas_1 = [
    (121925,8665891),
    (173929,8610218),
    (226807,8901836),
    (295067,8903704),
    (301725,8691456),
    (308125,8914793),
    (343464,8925363),
    (352221,8940701),
    (371704,8711768),
    (380254,8973864),
    (387920,8974436),
    (408546,8691041),
    (430112,8943488),
    (450206,8541310),
    (511553,8914590),
    (515582,8780815),
    (546254,8654704),
    (554127,8681821),
    (554358,8726328),
    (567336,8647181),
    (620526,8963839),
    (648290,8633139),
    (687369,8688575),
    (703777,8638908),
    (706218,8718020),
    (729374,8644723),
    (752450,8664392),
    (805780,8714269),
    (864793,8902765),
    (883033,8845041),
    #(258355,8765638),
    #(286698,8936504),
    #(427605,8880339),
    #(632914,8644813),
    #(796422,8676021),
]

In [ ]:
coordenadas_2 = [
    (115747,8707909),
    (261379,8730353),
    (268013,8715949),
    (268770,8712989),
    (280414,8698234),
    (296281,8698425),
    (299693,8689152),
    (422595,8752290),
    (437086,8880025),
    (440118,8327791),
    (440356,8911607),
    (440720,8905407),
    (447942,8911721),
    (450214,8908927),
    (510175,8914162),
    (511554,8914590),
    (515444,8927586),
    (520782,8921226),
    (522243,8673817),
    (522803,8713678),
    (535340,8683516),
    (550830,8663983),
    (560461,8657351),
    (567365,8695326),
    (572067,8871599),
    (647382,8872398),
    (738770,8666026),
    (751712,8542380),
    (76954,8649981),
    (406295,8672767),
    #(494821,8694145),
    #(506314,8935921),
    #(552716,8733620),
    #(719369,8539381),
    #(772971,8542561),
  ]

In [ ]:
coordenadas_3 = [
    (121925,8665892),
    (226807,8901837),
    (301725,8691457),
    (343464,8925364),
    (352221,8940702),
    (371704,8711769),
    (408546,8691042),
    (427605,8880340),
    (430112,8943489),
    (450206,8541311),
    (511553,8914591),
    (515582,8780816),
    (546254,8654705),
    (554127,8681822),
    (554358,8726329),
    (567336,8647182),
    (620526,8963840),
    (632914,8644814),
    (648290,8633140),
    (687369,8688576),
    (760354,8655081),
    (787201,8688286),
    (805780,8714270),
    (815955,8719312),
    (277156,8909926),
    (286338,8885363),
    (344259,8857689),
    (365497,8860499),
    (369520,8890190),
    (372909,8895488),
]

In [ ]:
coordenadas_4 = [
    (76954,8649982),
    (268770,8712990),
    (296281,8698426),
    (299693,8689153),
    (406295,8672768),
    (422595,8752291),
    (437086,8880026),
    (440118,8327792),
    (447942,8911722),
    (450214,8908928),
    (506314,8935922),
    (510175,8914163),
    (511554,8914591),
    (515444,8927587),
    (520782,8921227),
    (522243,8673818),
    (522803,8713679),
    (535340,8683517),
    (550830,8663984),
    (552716,8733621),
    (560461,8657352),
    (572067,8871600),
    (647382,8872399),
    (719369,8539382),
    (720187,8553464),
    (741719,8550648),
    (751712,8542381),
    (772971,8542562),
    (379667,8897033),
    (382756,8911213),
]

In [ ]:
coordenadas_5 = [
    (667630,8678655),
    (682620,8682562),
    (684256,8689557),
    (691433,8694690),
    (391433,8872576),
    (394908,8884602),
    (399405,8888520),
    (405470,8871383),
    (414600,8878436),
    (411159,8888247),
    (410478,8894312),
    (401347,8906576),
    (390854,8907053),
    (425809,8901398),
    (450747,8907122),
    (459810,8915060),
    (465363,8911039),
    (422334,8842118),
    (817908,8575903),
    (817363,8592870),
    (817295,8608678),
    (810686,8619512),
    (801555,8622169),
    (807892,8647312),
    (786428,8634434),
    (767350,8619171),
    (776003,8634366),
    (752291,8619784),
    (738936,8626325),
    (727898,8644450),
    (737029,8662370),
    (771097,8670956),
    (772460,8698075),
    (760604,8721378),
    (718768,8708432),
    (784861,8871349),
    (795900,8889644),
    (649028,8919846),
    (631244,8933883),
    (544607,8875352),
    (542938,8899166),
    (498784,8914021),
]

In [ ]:
coordenadas_6 = [
    (126572,8667337),
    (87848,8664731),
    (78671,8657259),
    (82060,8655804),
    (92594,8659474),
    (94071,8648766),
    (-39963,8861384),
    (-33722,8841517),
    (29727,8824042),
    (45330,8817766),
    (38846,8817316),
    (120897,8824146),
    (86433,8805024),
    (78459,8781725),
    (-170070,8829260),
    (-209042,8811335),
    (-256941,8830127),
    (-267360,8811335),
    (-301616,8901829),
    (-310579,8880557),
    (-331157,8853184),
    (-40258,8868110),
    (-42633,8860950),
    (-54976,8901309),
    (-34208,8933970),
    (-32249,8937974),
    (-30697,8961231),
    (-28253,8972152),
    (-6288,8983091),
    (-161151,8942612),
]

In [ ]:
coordenadas_7 = [
    (30429,8993545),
    (42842,8992817),
    (55567,8993337),
    (99219,8975099),
    (118791,8974163),
    (86650,9014105),
    (-306232,8925527),
    (-349482,8931047),
    (-343731,8875700),
    (-341046,8864598),
    (-357678,8829895),
    (-324353,8835185),
    (-227857,8855457),
    (-138762,8873196),
    (-104572,8887556),
    (-45082,8870018),
    (131016,8852058),
    (150645,8865091),
    (93850,8860908),
    (-192460,9041611),
    (-194290,9036181),
    (-185401,9021701),
    (-177196,9031757),
    (-176311,9042617),
    (-109942,8997707),
    (-97433,9001770),
    (-458599,8970758),
    (-449414,8950915),
    (-444842,8962673),
    (-442228,8961386),
]

In [ ]:
coordenadas_8 = [
    (-419421,8977556),
    (-327309,8988899),
    (-317374,8993363),
    (-316147,9007924),
    (-297102,9023390),
    (-248833,8984997),
    (-232060,8979164),
    (-240306,8967017),
    (-236123,8958208),
    (-232985,8953462),
    (-201832,8969813),
    (-194150,8966373),
    (-186950,8958610),
    (-192963,8953100),
    (-186045,8952014),
    (-183913,8834742),
    (-157466,8806727),
    (-154067,8789451),
    (-156963,8783699),
    (-136570,8756749),
    (-396219,8782103),
    (-357095,8752982),
    (-366614,8689697),
    (-369001,8685379),
    (-375410,8674787),
    (-255195,8672937),
    (-239495,8668244),
    (-251294,8678890),
    (-304932,8643973),
    (-293327,8648960),
]

In [ ]:
coordenadas_9 = [
    (-239495,8668243),
    (-84589,8659292),
    (-200631,8672652),
    (-184583,8676835),
    (227003,8997466),
    (220406,8994892),
    (259443,8993202),
    (324388,8915979),
    (343184,8823148),
    (-36503,8956788),
    (-104104,8903295),
    (358252,8885444),
    (369514,8882085),
    (365331,8859661),
    (368509,8852219),
    (-296565,8921484),
    (-377937,8907808),
    (-404304,8909095),
    (-443200,8906038),
    (-419749,8954829),
    (-418704,8967017),
    (-420805,8978943),
    (-427502,8992881),
    (-358700,8752827),
    (-257629,8666277),
    (-237779,8626878),
    (-219779,8619396),
    (-182884,8648297),
    (-145788,8648176),
    (-147839,8646547),
]

In [ ]:
coordenadas_10 = [
    (-239495,8668242),
    (-200634,8672652),
    (-184583,8676836),
    (69242,8980537),
    (82063,8975378),
    (197710,8944512),
    (233278,8945035),
    (239935,8944251),
    (241725,8939826),
    (245948,8947509),
    (246290,8946443),
    (252947,8950103),
    (222860,8994208),
    (231267,8990990),
    (238346,8979325),
    (253792,8998431),
    (301718,8986646),
    (372772,8973453),
    (368187,8984112),
    (348377,8960235),
    (372183,8962349),
    (389686,8951994),
    (386730,8959334),
    (343874,8927698),
    (398133,8922228),
    (396806,8928704),
    (403021,8938941),
    (384015,8906420),
    (364305,8895258),
    (361269,8889949),
]

In [ ]:
recortar_imagens(main_images, coordenadas_1, cropped_images)

Imagem recortada 0 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_121925L_8665891N.tif
Imagem recortada 1 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_173929L_8610218N.tif
Imagem recortada 2 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_226807L_8901836N.tif
Imagem recortada 3 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_295067L_8903704N.tif
Imagem recortada 4 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_301725L_8691456N.tif
Imagem recortada 5 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_308125L_8914793N.tif
Imagem recortada 6 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_343464L_8925363N.tif
Imagem recortada 7 salva em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/cropped_images/imagem_352221L_8940701N.tif
Imagem recortada 8 salva

In [ ]:
recortar_shapefile_por_imagens(shp_deforestation_total, cropped_images, shp_deforestation)

Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_121925L_8665891N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_173929L_8610218N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_226807L_8901836N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_295067L_8903704N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_301725L_8691456N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_308125L_8914793N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/shapefiles_deforestation/imagem_343464L_8925363N.shp
Shapefile recortado salvo em: /content/gdrive/MyDrive/TCC/Dataset_Ama

In [ ]:
shapefiles = [file for file in os.listdir(shp_deforestation) if file.endswith('.shp')]
image_files = [file for file in os.listdir(cropped_images) if file.endswith('.tif')]

In [ ]:
print(shapefiles)
print(image_files)
print(len(shapefiles))
print(len(image_files))

['imagem_368509L_8852219N.shp', 'imagem_-296565L_8921484N.shp', 'imagem_-377937L_8907808N.shp', 'imagem_-404304L_8909095N.shp', 'imagem_-443200L_8906038N.shp', 'imagem_-419749L_8954829N.shp', 'imagem_-418704L_8967017N.shp', 'imagem_-420805L_8978943N.shp', 'imagem_-427502L_8992881N.shp', 'imagem_-358700L_8752827N.shp', 'imagem_-237779L_8626878N.shp', 'imagem_-257629L_8666277N.shp', 'imagem_-182884L_8648297N.shp', 'imagem_-145788L_8648176N.shp', 'imagem_-239495L_8668242N.shp', 'imagem_-147839L_8646547N.shp', 'imagem_-200634L_8672652N.shp', 'imagem_-184583L_8676836N.shp', 'imagem_69242L_8980537N.shp', 'imagem_82063L_8975378N.shp', 'imagem_197710L_8944512N.shp', 'imagem_233278L_8945035N.shp', 'imagem_239935L_8944251N.shp', 'imagem_241725L_8939826N.shp', 'imagem_245948L_8947509N.shp', 'imagem_246290L_8946443N.shp', 'imagem_252947L_8950103N.shp', 'imagem_222860L_8994208N.shp', 'imagem_231267L_8990990N.shp', 'imagem_238346L_8979325N.shp', 'imagem_253792L_8998431N.shp', 'imagem_301718L_8986646

In [ ]:
for shapefile in shapefiles:
    base_name = os.path.splitext(shapefile)[0]  # Extrai o nome base do shapefile
    print(base_name)

    # Encontra um arquivo de imagem correspondente baseado no nome
    image_file = next((img for img in image_files if img.startswith(base_name)), None)

    if image_file:
        output_tif = path + f'/masks_tif/{base_name}.tif'
        output_png = path + f'/masks_png/{base_name}.png'
        shapefile_path = os.path.join(shp_deforestation, shapefile)
        image_path = os.path.join(cropped_images, image_file)

        print(f"Processando shapefile: {shapefile} e imagem: {image_file}")

        x_min, y_min, x_max, y_max = verifica_coordenada_vertice(image_path)

        # Rasterize usando as variáveis
        !gdal_rasterize -l {base_name} -a scene_id -tr 64.0 64.0 -a_nodata 0.0 -te {x_min} {y_min} {x_max} {y_max} -ot Byte -of GTiff '{shapefile_path}' '{output_tif}'
        !gdal_translate -of PNG '{output_tif}' '{output_png}'

        # Ajusta a máscara
        mask = Image.open(output_png)
        mask_array = np.array(mask)
        mask_array[mask_array != 0] = 255  # Converte fundo para branco
        mask_array[mask_array == 0] = 0
        adjusted_mask = Image.fromarray(mask_array)
        adjusted_mask.save(output_png)
    else:
        print(f"Nenhuma imagem correspondente encontrada para o shapefile: {shapefile}")

imagem_368509L_8852219N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_368509L_8852219N.shp
imagem_-296565L_8921484N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-296565L_8921484N.shp
imagem_-377937L_8907808N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-377937L_8907808N.shp
imagem_-404304L_8909095N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-404304L_8909095N.shp
imagem_-443200L_8906038N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-443200L_8906038N.shp
imagem_-419749L_8954829N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-419749L_8954829N.shp
imagem_-418704L_8967017N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-418704L_8967017N.shp
imagem_-420805L_8978943N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-420805L_8978943N.shp
imagem_-427502L_8992881N
Nenhuma imagem correspondente encontrada para o shapefile: imagem_-427502

In [ ]:
def excluir_imagens_nao_associadas(shp_deforestation, cropped_images):
    # Lista todos os shapefiles e extrai os nomes base
    shapefile_bases = {os.path.splitext(file)[0] for file in os.listdir(shp_deforestation) if file.endswith('.shp')}

    # Lista todas as imagens
    image_files = os.listdir(cropped_images)

    for image_file in image_files:
        # Extrai o nome base da imagem
        image_base = os.path.splitext(image_file)[0]

        # Se o nome base da imagem não estiver entre os nomes base dos shapefiles, exclui a imagem
        if image_base not in shapefile_bases:
            os.remove(os.path.join(cropped_images, image_file))
            print(f"Imagem excluída: {image_file}")

In [ ]:
def excluir_arquivos_aux_xml(diretorio):
    # Lista todos os arquivos no diretório
    arquivos = os.listdir(diretorio)

    # Itera sobre os arquivos
    for arquivo in arquivos:
        # Constrói o caminho completo do arquivo
        caminho_completo = os.path.join(diretorio, arquivo)

        # Verifica se o arquivo atual é um .aux.xml
        if arquivo.endswith('.aux.xml'):
            # Exclui o arquivo
            os.remove(caminho_completo)
            print(f"Arquivo excluído: {arquivo}")


In [ ]:
excluir_imagens_nao_associadas(shp_deforestation, cropped_images)

Imagem excluída: imagem_308125L_8914793N.tif.aux.xml
Imagem excluída: imagem_387920L_8974436N.tif.aux.xml
Imagem excluída: imagem_295067L_8903704N.tif.aux.xml


In [ ]:
diretorio_masks_png = '/content/gdrive/MyDrive/TCC/Dataset_Amazonia1_2.0/masks_png/'

In [ ]:
excluir_arquivos_aux_xml(diretorio_masks_png)

Arquivo excluído: imagem_308125L_8914793N.png.aux.xml
Arquivo excluído: imagem_226807L_8901836N.png.aux.xml
Arquivo excluído: imagem_295067L_8903704N.png.aux.xml
Arquivo excluído: imagem_301725L_8691456N.png.aux.xml
Arquivo excluído: imagem_343464L_8925363N.png.aux.xml
Arquivo excluído: imagem_352221L_8940701N.png.aux.xml
Arquivo excluído: imagem_371704L_8711768N.png.aux.xml
Arquivo excluído: imagem_380254L_8973864N.png.aux.xml
Arquivo excluído: imagem_387920L_8974436N.png.aux.xml
Arquivo excluído: imagem_408546L_8691041N.png.aux.xml
Arquivo excluído: imagem_430112L_8943488N.png.aux.xml
Arquivo excluído: imagem_450206L_8541310N.png.aux.xml
Arquivo excluído: imagem_511553L_8914590N.png.aux.xml
Arquivo excluído: imagem_515582L_8780815N.png.aux.xml
Arquivo excluído: imagem_546254L_8654704N.png.aux.xml
Arquivo excluído: imagem_554127L_8681821N.png.aux.xml
Arquivo excluído: imagem_554358L_8726328N.png.aux.xml
Arquivo excluído: imagem_567336L_8647181N.png.aux.xml
Arquivo excluído: imagem_620